In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Groupby技术

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 
                   'key2': ['one', 'two', 'one', 'two', 'one'], 
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [3]:
df

,data1,data2,key1,key2
0,0.732649,-0.355013,a,one
1,0.451556,0.043225,a,two
2,-0.853074,-0.438167,b,one
3,-0.246884,1.170596,b,two
4,1.468076,0.797548,a,one


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a    0.884094
b   -0.549979
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one     1.100363
      two     0.451556
b     one    -0.853074
      two    -0.246884
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,1.100363,0.451556
b,-0.853074,-0.246884


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005    0.451556
            2006   -0.853074
Ohio        2005    0.242883
            2006    1.468076
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.884094,0.161920
b,-0.549979,0.366214


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.100363  0.221268
     two   0.451556  0.043225
b    one  -0.853074 -0.438167
     two  -0.246884  1.170596

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 对分组进行迭代

In [16]:
for name, group in df.groupby('key1'):
    print name
    print group

a
      data1     data2 key1 key2
0  0.732649 -0.355013    a  one
1  0.451556  0.043225    a  two
4  1.468076  0.797548    a  one
b
      data1     data2 key1 key2
2 -0.853074 -0.438167    b  one
3 -0.246884  1.170596    b  two


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0  0.732649 -0.355013    a  one
4  1.468076  0.797548    a  one
a two
      data1     data2 key1 key2
1  0.451556  0.043225    a  two
b one
      data1     data2 key1 key2
2 -0.853074 -0.438167    b  one
b two
      data1     data2 key1 key2
3 -0.246884  1.170596    b  two


In [18]:
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,data1,data2,key1,key2
2,-0.853074,-0.438167,b,one
3,-0.246884,1.170596,b,two


In [20]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis = 1)

In [22]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  0.732649 -0.355013
 1  0.451556  0.043225
 2 -0.853074 -0.438167
 3 -0.246884  1.170596
 4  1.468076  0.797548, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

## 选取一个或者一组列

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.221268
     two   0.043225
b    one  -0.438167
     two   1.170596

In [24]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [25]:
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     one     0.221268
      two     0.043225
b     one    -0.438167
      two     1.170596
Name: data2, dtype: float64

## 通过字典或者Series进行分组

In [27]:
people = pd.DataFrame(np.random.randn(5,5), 
                      columns = ['a', 'b', 'c', 'd', 'e'], 
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [28]:
people.ix[2 : 3, ['b', 'c']] = np.nan

In [29]:
people

,a,b,c,d,e
Joe,-1.452857,-0.159133,-1.052526,0.404744,0.498541
Steve,0.138523,0.904831,-1.481551,-1.230443,1.286072
Wes,0.168501,NaN,NaN,0.909115,-1.093348
Jim,0.772743,0.466161,-0.606034,0.914322,0.617591
Travis,-0.126105,-0.770354,0.884357,-0.815101,-0.414483


In [30]:
mapping = {'a' : 'red', 'b' : 'red', 
           'c' : 'blue', 'd' : 'blue', 
           'e' : 'red', 'f' : 'orange'}

In [31]:
by_columns = people.groupby(mapping, axis = 1)

In [32]:
by_columns.sum()

,blue,red
Joe,-0.647781,-1.113448
Steve,-2.711994,2.329427
Wes,0.909115,-0.924847
Jim,0.308287,1.856495
Travis,0.069256,-1.310941


In [33]:
map_series = pd.Series(mapping)

In [34]:
by_columns = people.groupby(map_series, axis = 1).count()

In [35]:
by_columns

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 通过函数进行分组

In [36]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.511613,0.307028,-1.658560,2.228181,0.022783
5,0.138523,0.904831,-1.481551,-1.230443,1.286072
6,-0.126105,-0.770354,0.884357,-0.815101,-0.414483


In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [38]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.452857 -0.159133 -1.052526  0.404744 -1.093348
  two  0.772743  0.466161 -0.606034  0.914322  0.617591
5 one  0.138523  0.904831 -1.481551 -1.230443  1.286072
6 two -0.126105 -0.770354  0.884357 -0.815101 -0.414483

## 根据索引级别分组

In [39]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names = ['cty', 'tenor'])

In [40]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)

In [41]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.221990  0.635272  0.628668 -0.259629 -0.173119
1      1.766677  0.783901  0.854890  0.284011  0.257541
2      1.467777 -0.646064  0.612808 -0.518749  0.305536
3     -0.121749  0.600726  0.695772 -0.149847  1.228294

In [42]:
hier_df.groupby(level = 'cty', axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 数据聚合

In [43]:
df

,data1,data2,key1,key2
0,0.732649,-0.355013,a,one
1,0.451556,0.043225,a,two
2,-0.853074,-0.438167,b,one
3,-0.246884,1.170596,b,two
4,1.468076,0.797548,a,one


In [44]:
grouped = df.groupby('key1')

In [45]:
grouped['data1'].quantile(0.9)

key1
a    1.320991
b   -0.307503
Name: data1, dtype: float64

In [46]:
def peak2peak(arr):
    return arr.max() - arr.min()

In [47]:
grouped.agg(peak2peak)

,data1,data2
key1,,
a,1.01652,1.152561
b,0.60619,1.608762


In [48]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.884094  0.161920
     std    0.524909  0.585377
     min    0.451556 -0.355013
     25%    0.592102 -0.155894
     50%    0.732649  0.043225
     75%    1.100363  0.420386
     max    1.468076  0.797548
b    count  2.000000  2.000000
     mean  -0.549979  0.366214
     std    0.428641  1.137567
     min   -0.853074 -0.438167
     25%   -0.701526 -0.035976
     50%   -0.549979  0.366214
     75%   -0.398431  0.768405
     max   -0.246884  1.170596

In [49]:
tips = pd.read_csv('tips.csv')

In [50]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [51]:
tips[: 6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


## 面向列的多函数应用

In [52]:
grouped = tips.groupby(['sex', 'smoker'])

In [53]:
grouped_pct = grouped['tip_pct']

In [54]:
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [55]:
grouped_pct.agg(['mean', 'std', peak2peak])

mean       std  peak2peak
sex    smoker                               
Female No      0.156921  0.036421   0.195876
       Yes     0.182150  0.071595   0.360233
Male   No      0.160669  0.041849   0.220186
       Yes     0.152771  0.090588   0.674707

In [56]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [57]:
functions = ['count', 'mean', 'max']

In [58]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [59]:
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [60]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [61]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [62]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [63]:
grouped.agg({'tip': np.max, 'size': np.sum})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [64]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

## 以“无索引”的形式返回聚合数据

In [65]:
tips.groupby(['sex', 'smoker'], as_index = False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [66]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'}).reset_index()

sex smoker   tip_pct                               size
                       min       max      mean       std  sum
0  Female     No  0.056797  0.252672  0.156921  0.036421  140
1  Female    Yes  0.056433  0.416667  0.182150  0.071595   74
2    Male     No  0.071804  0.291990  0.160669  0.041849  263
3    Male    Yes  0.035638  0.710345  0.152771  0.090588  150